<a href="https://colab.research.google.com/github/yashica12/Neural-Networks/blob/master/Recommend_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install torch torchvision

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [0]:
from google.colab import drive

In [0]:
drive.mount('/content/drive/')


In [0]:
#importing data set 
path = '/content/drive/My Drive/Colab Notebooks/cnn/bm/bm/bm/'
movies = pd.read_csv(path + 'ml-1m/movies.dat',sep = '::' , header = None,engine='python' , encoding='latin-1')

In [0]:
users = pd.read_csv(path + 'ml-1m/users.dat',sep = '::' , header = None,engine='python' , encoding='latin-1')

In [0]:
ratings = pd.read_csv(path + 'ml-1m/ratings.dat',sep = '::' , header = None,engine='python' , encoding='latin-1')

In [0]:
#importing test and training set
training_set = pd.read_csv(path + 'ml-100k/u1.base' , delimiter='\t')

In [0]:
training_set =np.array(training_set,dtype='int')

In [0]:
test_set = pd.read_csv(path + 'ml-100k/u1.test' , delimiter='\t')
test_set = np.array(test_set,dtype='int')

In [0]:
# total users and movies in the data set 
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))



In [0]:
#converting data into an array 
def convert(data) : 
  new_data = []
  for i in range(1,nb_users + 1 ):
    id_movies = data[:,1][data[:,0] == i]
    print("MOVIES" , id_movies)
    id_ratings = data[:,2][data[:,0] == i]
    print("RATINGS" , id_ratings) 
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1 ] = id_ratings
    new_data.append(list(ratings))
  return new_data

  

In [0]:
training_set = convert(training_set)


In [0]:
test_set = convert(test_set)

In [38]:
len(test_set)

943

In [0]:
training_set = torch.FloatTensor(training_set)

In [0]:
test_set = torch.FloatTensor(test_set)

In [0]:
#converting liked = 1 , not liked =0 , not rated = -1 
training_set[training_set == 0 ] = -1 
training_set[training_set == 1 ] = 0 
training_set[training_set == 2 ] = 0 
training_set[training_set >= 3 ] = 1

test_set[test_set == 0 ] = -1 
test_set[test_set == 1 ] = 0 
test_set[test_set == 2 ] = 0 
test_set[test_set >= 3 ] = 1

In [0]:
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

# Training the RBM
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))
  
    
      
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))  
  